In [41]:
# เชื่อมกับ duckdb
import duckdb
import pandas as pd

In [2]:
con = duckdb.connect(database=':memory:', read_only= False)

In [5]:
query = "select * from 'shopping_trends.csv'"

In [6]:
result = con.execute(query).fetchdf()

In [42]:
# โหลด dataset
df = pd.read_csv("shopping_trends.csv")

# เชื่อมกับ duckdb
con = duckdb.connect()
con.register("shopping", df)

In [13]:
result.head(10)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,True,Credit Card,Express,True,True,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,True,Bank Transfer,Express,True,True,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,True,Cash,Free Shipping,True,True,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,True,PayPal,Next Day Air,True,True,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,True,Cash,Free Shipping,True,True,31,PayPal,Annually
5,6,46,Male,Sneakers,Footwear,20,Wyoming,M,White,Summer,2.9,True,Venmo,Standard,True,True,14,Venmo,Weekly
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,3.2,True,Debit Card,Free Shipping,True,True,49,Cash,Quarterly
7,8,27,Male,Shorts,Clothing,34,Louisiana,L,Charcoal,Winter,3.2,True,Debit Card,Free Shipping,True,True,19,Credit Card,Weekly
8,9,26,Male,Coat,Outerwear,97,West Virginia,L,Silver,Summer,2.6,True,Venmo,Express,True,True,8,Venmo,Annually
9,10,57,Male,Handbag,Accessories,31,Missouri,M,Pink,Spring,4.8,True,PayPal,2-Day Shipping,True,True,4,Cash,Quarterly


In [8]:
result.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Payment Method',
       'Shipping Type', 'Discount Applied', 'Promo Code Used',
       'Previous Purchases', 'Preferred Payment Method',
       'Frequency of Purchases'],
      dtype='object')

In [43]:
#หาลูกค้าที่อายุ 18–30 ปี และซื้อสินค้าประเภท Coat	มากที่สุด
query = """
SELECT "Customer ID", COUNT(*) AS purchase_count
FROM shopping
WHERE "Age" BETWEEN 18 AND 30
  AND "Item Purchased" = 'Coat'
GROUP BY "Customer ID"
ORDER BY purchase_count DESC
LIMIT 5;
"""
customer_result1 = con.execute(query).fetchdf()
customer_result1

,Customer ID,purchase_count
0,9,1
1,374,1
2,429,1
3,552,1
4,768,1


In [44]:
# Scenario 2: หาสีเสื้อผ้าที่ขายดีที่สุด (ยอดขายรวมมากที่สุด)
query2 = """
SELECT "Color", COUNT(*) AS total_sold
FROM shopping
GROUP BY "Color"
ORDER BY total_sold DESC
LIMIT 5;
"""
result2 = con.execute(query2).fetchdf()
result2

,Color,total_sold
0,Olive,177
1,Yellow,174
2,Silver,173
3,Teal,172
4,Green,169


In [45]:
# Scenario 3: หาประเภทสินค้าที่เพศชายกับเพศหญิงนิยมซื้อมากที่สุด
query3 = """
SELECT "Gender", "Category", COUNT(*) AS count_purchase
FROM shopping
GROUP BY "Gender", "Category"
ORDER BY "Gender", count_purchase DESC;
"""
result3 = con.execute(query3).fetchdf()
result3

,Gender,Category,count_purchase
0,Female,Clothing,556
1,Female,Accessories,392
2,Female,Footwear,199
3,Female,Outerwear,101
4,Male,Clothing,1181
5,Male,Accessories,848
6,Male,Footwear,400
7,Male,Outerwear,223


In [55]:
# Scenario 4 : ดูประเภทสินค้าที่แต่ละกลุ่มอายุนิยม
query4 = """
SELECT
  CASE
    WHEN "Age" BETWEEN 18 AND 25 THEN '18-25'
    WHEN "Age" BETWEEN 26 AND 35 THEN '26-35'
    WHEN "Age" BETWEEN 36 AND 50 THEN '36-50'
    ELSE '51+'
  END AS age_group,
  "Category",
  COUNT(*) AS purchase_count
FROM shopping
GROUP BY age_group, "Category"
ORDER BY age_group, purchase_count DESC;
"""
result4 = con.execute(query4).fetchdf()
result4

,age_group,Category,purchase_count
0,18-25,Clothing,269
1,18-25,Accessories,172
2,18-25,Footwear,79
3,18-25,Outerwear,51
4,26-35,Clothing,337
5,26-35,Accessories,243
6,26-35,Footwear,108
7,26-35,Outerwear,54
8,36-50,Clothing,480
9,36-50,Accessories,356


In [49]:
# Scenario 5: หาค่าเฉลี่ยยอดซื้อของลูกค้าแบ่งตามเพศ
query5 = """
SELECT "Gender", AVG("Purchase Amount (USD)") AS avg_spent
FROM shopping
GROUP BY "Gender";
"""
result5 = con.execute(query5).fetchdf()
result5

,Gender,avg_spent
0,Male,59.536199
1,Female,60.249199


In [50]:
# Scenario 6: หาฤดูกาลที่มียอดขายสูงสุด
query6 = """
SELECT "Season", SUM("Purchase Amount (USD)") AS total_sales
FROM shopping
GROUP BY "Season"
ORDER BY total_sales DESC;
"""
result6 = con.execute(query6).fetchdf()
result6

,Season,total_sales
0,Fall,60018.0
1,Spring,58679.0
2,Winter,58607.0
3,Summer,55777.0


In [51]:
# Scenario 7: หาลูกค้าที่ซื้อบ่อยที่สุด (จาก Frequency of Purchases)
query7 = """
SELECT "Customer ID", MAX("Frequency of Purchases") AS freq
FROM shopping
GROUP BY "Customer ID"
ORDER BY freq DESC
LIMIT 5;
"""
result7 = con.execute(query7).fetchdf()
result7

,Customer ID,freq
0,15,Weekly
1,14,Weekly
2,54,Weekly
3,30,Weekly
4,60,Weekly


In [52]:
# Scenario 8: เปรียบเทียบการใช้จ่ายเฉลี่ยของลูกค้าแบบสมัครสมาชิก vs ไม่สมัคร
query8 = """
SELECT "Subscription Status", AVG("Purchase Amount (USD)") AS avg_spent
FROM shopping
GROUP BY "Subscription Status";
"""
result8 = con.execute(query8).fetchdf()
result8

,Subscription Status,avg_spent
0,No,59.865121
1,Yes,59.491928


In [53]:
# Scenario 9: หาวิธีจ่ายเงินที่ลูกค้านิยมใช้มากที่สุด
query9 = """
SELECT "Payment Method", COUNT(*) AS total_used
FROM shopping
GROUP BY "Payment Method"
ORDER BY total_used DESC;
"""
result9 = con.execute(query9).fetchdf()
result9

,Payment Method,total_used
0,Credit Card,696
1,Venmo,653
2,Cash,648
3,PayPal,638
4,Debit Card,633
5,Bank Transfer,632


In [54]:
# Scenario 10: หาลูกค้าที่เคยซื้อเกิน 10 ครั้ง และยังซื้อซ้ำอยู่เรื่อย ๆ
query10 = """
SELECT "Customer ID", "Previous Purchases", "Frequency of Purchases"
FROM shopping
WHERE "Previous Purchases" > 10
ORDER BY "Frequency of Purchases" DESC;
"""
result10 = con.execute(query10).fetchdf()
result10

,Customer ID,Previous Purchases,Frequency of Purchases
0,2203,23,Weekly
1,1336,26,Weekly
2,128,26,Weekly
3,3031,23,Weekly
4,6,14,Weekly
...,...,...,...
3111,538,14,Annually
3112,525,44,Annually
3113,1934,28,Annually
3114,5,31,Annually
